In [1]:
from datetime import datetime, timedelta
import pandas as pd
import pytz
def get_time_ranges(interval, old_start_time, old_end_time, date_format = "%H:%M:%S.%f", delta_period_type="minutes"):
    start_date = old_start_time
    end_date = old_end_time+ timedelta(seconds = 1)
    dates = []
    while start_date < end_date:
        delta_period_arg = {delta_period_type: interval}
        interval_end = start_date + timedelta(**delta_period_arg)
        if interval_end.time() < start_date.time():
            dates.append({"start_date" : start_date.time(), "end_date" : datetime.max.time()})
            if interval_end > end_date:
                interval_end = end_date
            interval_end = interval_end - timedelta(microseconds=1)
            dates.append({"start_date" : datetime.min.time(), "end_date" : interval_end.time()})
            start_date = start_date + timedelta(**delta_period_arg)
            continue
        if interval_end > end_date:
            interval_end = end_date
        interval_end = interval_end - timedelta(microseconds=1)
        dates.append({"start_date" : start_date.time(), "end_date" : interval_end.time()})
        start_date = start_date + timedelta(**delta_period_arg)
    return dates

In [62]:
def calculate_status_trend(store_id)-> list():
    src1 = pd.read_csv("/Users/crest/Downloads/store status.csv")
    src2 = pd.read_csv("/Users/crest/Downloads/Menu hours.csv")
    src3 = pd.read_csv("/Users/crest/Downloads/bq-results-20230125-202210-1674678181880.csv")
    filtered_status = src1.query(f"store_id == {store_id}").reset_index()
    filtered_timezone = src3.query(f"store_id == {store_id}").reset_index()
    intervals = list()
    trends = list() * 7
    # print(datetime.strptime(filtered_status["timestamp_utc"][0], "%Y-%m-%d %H:%M:%S.%f UTC").time())

    local = None

    if filtered_timezone.empty:
        local = pytz.timezone("America/Chicago")
    else:
        local = pytz.timezone(filtered_timezone["timezone_str"][0])

    for day in range(7):
        start = None
        end = None

        filtered_time_range = src2.query(f"store_id == {store_id} and day == {day}").reset_index()

        trends.append([])
        intervals.append([])
        if filtered_time_range.empty:
            start = datetime.combine(datetime.today(), datetime.min.time())
            end = datetime.combine(datetime.today(), datetime.max.time())

        else:
            start = datetime.strptime(filtered_time_range["start_time_local"][0], "%H:%M:%S")
            end = datetime.strptime(filtered_time_range["end_time_local"][0], "%H:%M:%S")


        new_start = start
        local_dt = local.localize(new_start, is_dst=None)
        start = local_dt.astimezone(pytz.utc)
        

        new_end = end
        local_dt = local.localize(new_end, is_dst=None)
        end = local_dt.astimezone(pytz.utc)   
        # print("end") 
        # print(f"{end}  {new_end}")


        # print(f"{end}", end= "\n\n")
        interval = get_time_ranges(15, start, end)
        intervals.append(interval)
        # print(interval)
        # print(f"start {start} end {end}")
        # print((end - start).total_seconds()/60)
        max_interval_trend = [0] * len(interval)
        for _, row in filtered_status.iterrows():
            for index in range(len(interval)):
                curr_time = datetime.strptime(row["timestamp_utc"], "%Y-%m-%d %H:%M:%S.%f UTC").time()
                if(interval[index]["start_date"] > interval[index]["end_date"]):
                    if(curr_time < interval[index]["start_date"] 
                        and curr_time > interval[index]["end_date"]):
                        continue
                    if row["status"] == "active":
                        max_interval_trend[index] += 1
                    else:
                        max_interval_trend[index] -= 1
                    break
                else:
                    if(curr_time >= interval[index]["start_date"] 
                        and curr_time < interval[index]["end_date"]):
                        if row["status"] == "active":
                            max_interval_trend[index] += 1
                        elif row["status"] == "inactive":
                            max_interval_trend[index] -= 1
                        else:
                            continue
                        break
        
        for index in range(len(max_interval_trend)):
            if not max_interval_trend[index]:
                if not index:
                    max_interval_trend[index] = 1
                else:
                    max_interval_trend[index] = max_interval_trend[index-1]
        trends[day] = (max_interval_trend)

    return trends, intervals

In [47]:
def generate_ranges(start, end):
    ranges = list()
    while(start < end):
        new_end = datetime.combine(start, datetime.max.time())
        if new_end > end:
            new_end = end
        ranges.append({"start" : start, "end" : new_end})
        start = new_end + timedelta(microseconds= 1)
    return ranges

In [96]:
def calculate_uptime_and_downtime(ranges, trends, interval):
    uptime = 0
    downtime = 0
    for range in ranges:
        day = range["start"].weekday()
        index = 0
        for time in interval[day]:
            if range["start"].time() > time["end_date"]:
                continue
            if range["start"].time() <= time["start_date"]:
                range["start"] = datetime.combine(range["start"], time["start_date"])
            
            if range["start"].time() >= time["start_date"]:
                if range["end"].time() <= time["end_date"]:
                    if index < len(trends[day]) and trends[day][index] < 0:
                        downtime += (range["end"] - range["start"]).total_seconds()
                    else:
                        uptime += (range["end"] - range["start"]).total_seconds()
                    break
                else:
                    dt = datetime.combine(range["start"], time["end_date"])
                    if dt < range["start"]:
                        dt = dt + timedelta(days = 1)
                    if index < len(trends[day]) and trends[day][index] < 0:
                        downtime += (dt - range["start"]).total_seconds() 
                    else:
                        uptime += (dt - range["start"]).total_seconds()
                    if index < len(interval[day]) - 1:
                        range["start"] = datetime.combine(range["start"], interval[day][index+1]["start_date"])
            index += 1
    return uptime, downtime
                    
            

In [100]:

stores = set()
src1 = Students.object.filter(store_id = "dsdasd")
src1.groupby("storeId")
src1 = pd.read_csv("/Users/crest/Downloads/store status.csv")
for store in src1["store_id"]:
    stores.add(store)

end = datetime.today().utcnow()
start = end - timedelta(days= 6)
ranges_week = generate_ranges(start, end)
ranges_hour = generate_ranges(end - timedelta(hours= 1), end)
ranges_day = generate_ranges(end - timedelta(days = 1), end)

for store_id in stores:
    trends, interval = calculate_status_trend(s
                                              tore_id)
    uptime_week, downtime_week = calculate_uptime_and_downtime(ranges_week, trends, interval)
    uptime_day, downtime_day = calculate_uptime_and_downtime(ranges_day, trends, interval)
    uptime_hour, downtime_hour = calculate_uptime_and_downtime(ranges_hour, trends, interval)
    print(f"{(uptime_hour) / 60}  {(downtime_hour)/60}")
    print(f"{(uptime_day)/3600}   {(downtime_day)/3600}")
    print(f"{(uptime_week)/3600}   {(downtime_week)/3600}")


    


        



[{'start': datetime.datetime(2023, 8, 23, 11, 46, 36, 723001), 'end': datetime.datetime(2023, 8, 23, 12, 46, 36, 723001)}]


14092

In [6]:
import pandas as pd
src1 = pd.read_csv("/Users/crest/Downloads/bq-results-20230125-202210-1674678181880.csv")
src1.columns.tolist()

['store_id', 'day', 'start_time_local', 'end_time_local']